In [1]:
from openpyxl import Workbook

In [2]:
import requests

In [3]:
from bs4 import BeautifulSoup

In [4]:
import random

In [5]:
import string

In [6]:
import re

In [7]:
wb = Workbook()

In [8]:
ws = wb.active

In [9]:
ws1 = wb.create_sheet("Giełda")

In [10]:
ws2 = wb.create_sheet("Linki")

In [11]:
ws3 = wb.create_sheet("Filmweb")

In [12]:
print(wb.sheetnames)


['Sheet', 'Giełda', 'Linki', 'Filmweb']


In [13]:
url = "https://stooq.pl/q/?s=cdr"
r = requests.get(url)
data = r.text
site = BeautifulSoup(data)
random_word = random.choice(string.ascii_letters)

In [14]:
index = 0
url3 = "https://stooq.pl/q/?s="
valid_codes = []
while index < 5:
    random_word = random.choice(string.ascii_letters) + random.choice(string.ascii_letters) + random.choice(string.ascii_letters) 
    req_comp = requests.get(url3+random_word)
    data4 = req_comp.text
    site = BeautifulSoup(data4)
    if len(site.body.findAll('span', id='c2')) != 0:
        link_content = site.find('font',id='f16').find('a',href=True)
        if link_content.text == 'symbol waloru':
            print("Kod nie znaleziony.")
        else:
            valid_code = link_content['href'][5:]
            valid_codes.append(valid_code)
            print(f'Znaleziono kod: {valid_code}')
            index = index + 1
    else:
        valid_codes.append(random_word)
        index = index + 1
        
ws1.cell(row=1, column=1).value = 'Kod'
ws1.cell(row=1, column=2).value = 'Kurs'
ws1.cell(row=1, column=3).value = 'Zmiana procentowa'
ws1.cell(row=1, column=4).value = 'Transakcje'
row = 2
        
for code in valid_codes:
    print(code)
    params = {"s": code}
    response = requests.get("http://stooq.pl/q/", params=params)
    soup = BeautifulSoup(response.content, 'html.parser')
    price = soup.find(text="Kurs").parent.find("span").text
    percent_change = soup.find(text="Zmiana").parent.findAll("span")[1].text
    transactions = soup.find(text="Transakcje").parent.find("span").text
    print(f"{price}, {percent_change}, {transactions}")
    ws1.cell(row=row, column=1).value = code
    ws1.cell(row=row, column=2).value = price if price else 'Brak danych'
    ws1.cell(row=row, column=3).value =  percent_change if percent_change else 'Brak danych'
    ws1.cell(row=row, column=4).value =  transactions if transactions else 'Brak danych'
    row += 1

Znaleziono kod: pffd.us
Znaleziono kod: akm20.f
Kod nie znaleziony.
Znaleziono kod: qqxt.us
Znaleziono kod: omi.uk
Znaleziono kod: gtls.us
pffd.us
24.954, (+0.02%), 55
akm20.f
1.495, (+0.61%), 
qqxt.us
56.1100, (+0.02%), 2
omi.uk
2.90, (0.00%), 2
gtls.us
55.7900, (-1.71%), 263


In [15]:
urlik = "https://www.filmweb.pl/film/To%3A+Rozdzia%C5%82+2-2019-793838"
r2 = requests.get(urlik)
data2 = r2.text
site = BeautifulSoup(data2)

filmweb = {}
filmweb['Director'] = site.find(text="reżyseria:").next_element.find("a").text
filmweb['Year'] = site.find("h1", class_="filmTitle").find('span').text[1:5]
boxoffice = site.find(text="boxoffice:").next_element.text
filmweb['BoxOffice']  = re.sub(r'(top #\d+)', '', boxoffice)
filmweb['Raiting'] = site.find("span", itemprop="ratingValue").text

row = 1
for key, value in filmweb.items():
    ws3.cell(row,column=1).value = key
    ws3.cell(row,column=2).value = value
    row += 1

In [16]:
urlik3 = "http://google.pl"
r3 = requests.get(urlik3)
data3 = r3.text
site = BeautifulSoup(data3)
row_row = 1
for link in site.find_all('a'):
    ws2.cell(row=row_row,column=1).value = link.get('href')
    row_row = row_row + 1

In [17]:
row_row = 1
for link in site.find_all('a'):
    ws2.cell(row=row_row,column=1).value = link.get('href')
    row_row = row_row + 1

In [19]:
wb.save("Lewandowski-165ICA2.xlsx")